In [1]:
import pandas as pd
train_2022 = pd.read_csv('train_2022.csv')
data_augmentation_chatGPT = pd.read_csv('data_augmentation_chatGPT.csv')
data_augmentation_random_2_words = pd.read_csv('data_augmentation_random_2_words.csv')
data_augmentation_random_3_words = pd.read_csv('data_augmentation_random_3_words.csv')
translated_en_data = pd.read_csv('translated_en_data.csv')

In [2]:
merged_data = pd.concat([train_2022], ignore_index=True)
merged_data = merged_data[:1000]
merged_data

,row_id,TEXT,LABEL
0,0,director dirk shafer and co-writer greg hinton...,0
1,1,"a charming , quirky and leisurely paced scotti...",1
2,2,"the price was good , and came quickly though ...",1
3,3,i was looking forward to this game for a coupl...,0
4,4,arguably the year 's silliest and most incoher...,0
...,...,...,...
995,995,"painful to watch , but viewers willing to take...",1
996,996,a byzantine melodrama that stimulates the high...,1
997,997,he 's super spy !,1
998,998,but this stuff will fall apart after only a fe...,0


In [ ]:
%%time
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.nn.utils.rnn import pad_sequence

# 確認是否有可用的 CUDA 設備，並設定使用的設備
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

# 載入預訓練的 tokenizer 和模型，這裡指定了 assemblyai 提供的 BERT large 模型
tokenizer = AutoTokenizer.from_pretrained('assemblyai/bert-large-uncased-sst2')
model = AutoModelForSequenceClassification.from_pretrained('assemblyai/bert-large-uncased-sst2')
model.to(device)

# 轉換文本為 token IDs
def tokenize_text(text):
    return tokenizer(text, return_tensors='pt', padding=True, truncation=True)['input_ids'][0]

merged_data['input_ids'] = merged_data['TEXT'].apply(tokenize_text)

# 轉換成可以輸入模型的格式，並移動到 CUDA 設備上
inputs = pad_sequence(merged_data['input_ids'].tolist(), batch_first=True).to(device)
labels = torch.tensor(merged_data['LABEL'].tolist()).to(device)

# 將資料拆分為訓練集和測試集
train_inputs, test_inputs, train_labels, test_labels = train_test_split(inputs, labels, test_size=0.2, random_state=42)

# 創建 PyTorch DataLoader
train_dataset = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

# 定義 optimizer 和損失函數
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# 訓練模型
model.train()
for epoch in range(3):  # 進行三個 epoch 的訓練
    for batch in train_loader:
        b_input_ids, b_labels = batch
        model.zero_grad()
        outputs = model(b_input_ids, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
# 評估模型
model.eval()
with torch.no_grad():
    test_outputs = model(test_inputs)
    predicted_labels = torch.argmax(test_outputs.logits, dim=1)

# 產生分類報告
report = classification_report(test_labels.cpu(), predicted_labels.cpu())
print(report)

Epoch 1, Loss: 0.4950263500213623
